In [3]:
#	地点名 (no)	地点番号
#	稚内	47401
#	北見枝幸	47402
#	羽幌	47404
#	雄武	47405
#	留萌	47406
#	旭川	47407
#	網走	47409
#	小樽	47411
#	札幌	47412
#	岩見沢	47413
#	帯広	47417
#	釧路	47418
#	根室	47420
#	寿都	47421
#	室蘭	47423
#	苫小牧	47424
#	浦河	47426
#	江差	47428
#	函館	47430
#	倶知安	47433
#	紋別	47435
#	広尾	47440
#	大船渡	47512
#	新庄	47520
#	若松	47570
#	深浦	47574
#	青森	47575
#	むつ	47576
#	八戸	47581
#	秋田	47582
#	盛岡	47584
#	宮古	47585
#	酒田	47587
#	山形	47588
#	仙台	47590
#	石巻	47592
#	福島	47595
#	白河	47597
#	小名浜	47598
#	輪島	47600
#	相川	47602
#	新潟	47604
#	金沢	47605
#	伏木	47606
#	富山	47607
#	長野	47610
#	高田	47612
#	宇都宮	47615
#	福井	47616
#	高山	47617
#	松本	47618
#	諏訪	47620
#	軽井沢	47622
#	前橋	47624
#	熊谷	47626
#	水戸	47629
#	敦賀	47631
#	岐阜	47632
#	名古屋	47636
#	飯田	47637
#	甲府	47638
#	富士山	47639
#	河口湖	47640
#	秩父	47641
#	つくば（館野）	47646
#	銚子	47648
#	上野	47649
#	津	47651
#	伊良湖	47653
#	浜松	47654
#	御前崎	47655
#	静岡	47656
#	三島	47657
#	東京	47662
#	尾鷲	47663
#	石廊崎	47666
#	網代	47668
#	横浜	47670
#	館山	47672
#	勝浦	47674
#	大島	47675
#	三宅島	47677
#	八丈島	47678
#	千葉	47682
#	四日市	47684
#	奥日光（日光）	47690
#	西郷	47740
#	松江	47741
#	境	47742
#	米子	47744
#	鳥取	47746
#	豊岡	47747
#	舞鶴	47750
#	伊吹山	47751
#	萩	47754
#	浜田	47755
#	津山	47756
#	京都	47759
#	彦根	47761
#	下関	47762
#	広島	47765
#	呉	47766
#	福山	47767
#	岡山	47768
#	姫路	47769
#	神戸	47770
#	大阪	47772
#	洲本	47776
#	和歌山	47777
#	潮岬	47778
#	奈良	47780
#	山口	47784
#	厳原	47800
#	平戸	47805
#	福岡	47807
#	飯塚	47809
#	佐世保	47812
#	佐賀	47813
#	日田	47814
#	大分	47815
#	長崎	47817
#	雲仙岳	47818
#	熊本	47819
#	阿蘇山	47821
#	延岡	47822
#	阿久根	47823
#	人吉	47824
#	鹿児島	47827
#	都城	47829
#	宮崎	47830
#	枕崎	47831
#	油津	47835
#	屋久島	47836
#	種子島	47837
#	牛深	47838
#	福江	47843
#	松山	47887
#	多度津	47890
#	高松	47891
#	宇和島	47892
#	高知	47893
#	剣山	47894
#	徳島	47895
#	宿毛	47897
#	清水	47898
#	室戸岬	47899
#	名瀬	47909
#	与那国島	47912
#	石垣島	47918
#	宮古島	47927
#	久米島	47929
#	那覇	47936
#	名護	47940
#	沖永良部	47942
#	南大東（南大東島）	47945
#	父島	47971
#	南鳥島	47991

In [27]:
import requests
import json
import pprint
import pandas as pd
import folium 
from sentinelsat import SentinelAPI, read_geojson, geojson_to_wkt
import geopandas as gpd
import json
import sys
import datetime
import csv
from tkinter import filedialog
import numpy as np

In [28]:
#AOIのGeojsonを開く

filename = filedialog.askopenfilename(
    title = "Geojsonを開く",
    filetypes = [("GEOJSONファイル",".geojson")]
    )

with open(str(filename)) as f:
    j = json.load(f)

for feature in j["features"]:
    coordinates = feature["geometry"]["coordinates"]
    point = [p for i in coordinates for j in i for p in j]

x = np.array(point)
coords = np.mean(x, axis=0)

print(filename)
print(coords)

footprint_geojson = geojson_to_wkt(read_geojson(str(filename)))

m = folium.Map(location=[coords[1],coords[0]], zoom_start=10)
folium.GeoJson(str(filename)).add_to(m)
m

C:/Users/Shingo Yoshino/Desktop/案件データ/A_LDM/05_長野市/01_PreDoc/NaganoSquare.geojson
[138.06942763  36.68845323]


In [35]:
#衛星データ

user = 'homodeus'
password = '74250815jY'
area = '47610' #In1をみて地点番号を入力してください。
api = SentinelAPI(user, password, 'https://scihub.copernicus.eu/dhus')

#YYYY M D h m s
start = datetime.datetime(2017, 4, 1, 0, 0, 0)
end = datetime.datetime(2022, 4, 1, 0, 0, 0)

products = api.query(footprint_geojson,
                     platformname = 'Sentinel-1',
                     #orbitdirection = 'Ascending',
                     #producttype = 'SLC',
                     date = (start, end)
                     )

products_gdf = api.to_geodataframe(products)
products_gdf_sorted = products_gdf.sort_values(['title'])
products_gdf_sorted

Querying products:   4%|3         | 100/2510 [00:00<?, ?product/s]

,title,link,link_alternative,link_icon,summary,ondemand,beginposition,endposition,ingestiondate,missiondatatakeid,...,platformname,platformidentifier,instrumentname,instrumentshortname,productclass,polarisationmode,acquisitiontype,uuid,productconsolidation,geometry
9220ce67-f47d-4118-bb7f-550a461da240,S1A_IW_GRDH_1SDV_20170412T205042_20170412T2051...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,"Date: 2017-04-12T20:50:42.321Z, Instrument: SA...",false,2017-04-12 20:50:42.321,2017-04-12 20:51:11.340,2017-04-13 01:02:15.303,109011,...,Sentinel-1,2014-016A,Synthetic Aperture Radar (C-band),SAR-C SAR,S,VV VH,NOMINAL,9220ce67-f47d-4118-bb7f-550a461da240,NaN,"POLYGON ((139.30930 36.65808, 136.51891 37.053..."
de1f0f99-3785-4b98-8912-cd8b869ad91b,S1A_IW_GRDH_1SDV_20170412T205111_20170412T2051...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,"Date: 2017-04-12T20:51:11.342Z, Instrument: SA...",false,2017-04-12 20:51:11.342,2017-04-12 20:51:36.340,2017-04-13 01:02:15.488,109011,...,Sentinel-1,2014-016A,Synthetic Aperture Radar (C-band),SAR-C SAR,S,VV VH,NOMINAL,de1f0f99-3785-4b98-8912-cd8b869ad91b,NaN,"POLYGON ((138.95596 35.15300, 136.21970 35.549..."
b6e2a237-f5e4-4f3f-9160-ac0d390c49cc,S1A_IW_GRDH_1SDV_20170424T205042_20170424T2051...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,"Date: 2017-04-24T20:50:42.846Z, Instrument: SA...",false,2017-04-24 20:50:42.846,2017-04-24 20:51:11.865,2017-04-25 01:20:09.146,110387,...,Sentinel-1,2014-016A,Synthetic Aperture Radar (C-band),SAR-C SAR,S,VV VH,NOMINAL,b6e2a237-f5e4-4f3f-9160-ac0d390c49cc,NaN,"POLYGON ((139.30902 36.65814, 136.51863 37.053..."
e7fd92a6-a70f-450b-91c0-e09c32ee5e91,S1A_IW_GRDH_1SDV_20170424T205111_20170424T2051...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,"Date: 2017-04-24T20:51:11.867Z, Instrument: SA...",false,2017-04-24 20:51:11.867,2017-04-24 20:51:36.865,2017-04-25 01:20:11.356,110387,...,Sentinel-1,2014-016A,Synthetic Aperture Radar (C-band),SAR-C SAR,S,VV VH,NOMINAL,e7fd92a6-a70f-450b-91c0-e09c32ee5e91,NaN,"POLYGON ((138.95575 35.15305, 136.21959 35.549..."
d5fde1fd-1719-4987-a3f6-5b09eb1ad856,S1A_IW_GRDH_1SDV_20170506T205043_20170506T2051...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,"Date: 2017-05-06T20:50:43.657Z, Instrument: SA...",false,2017-05-06 20:50:43.657,2017-05-06 20:51:12.681,2017-05-07 00:52:11.698,111737,...,Sentinel-1,2014-016A,Synthetic Aperture Radar (C-band),SAR-C SAR,S,VV VH,NOMINAL,d5fde1fd-1719-4987-a3f6-5b09eb1ad856,NaN,"POLYGON ((139.30919 36.65806, 136.51869 37.053..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
d660452d-c580-4150-a787-38e801681f1f,S1B_IW_SLC__1SDV_20211130T084211_20211130T0842...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,"Date: 2021-11-30T08:42:11.624Z, Instrument: SA...",false,2021-11-30 08:42:11.624,2021-11-30 08:42:38.578,2021-11-30 11:39:54.756,233230,...,Sentinel-1,2016-025A,Synthetic Aperture Radar (C-band),SAR-C SAR,S,VV VH,NOMINAL,d660452d-c580-4150-a787-38e801681f1f,NaN,"MULTIPOLYGON (((137.64059 36.75822, 140.44482 ..."
e3f8a4c3-ba6f-48e2-b2a2-e0a6d8c0d566,S1B_IW_SLC__1SDV_20211205T085008_20211205T0850...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,"Date: 2021-12-05T08:50:08.389Z, Instrument: SA...",false,2021-12-05 08:50:08.389,2021-12-05 08:5

In [36]:
#気象データ呼び出し関数

def api_call(area,dt):
    url = 'https://api.cultivationdata.net/past?no=' + str(area) + '&year=' + str(dt.year) + '&month=' + str(dt.month)
    r = requests.get(url, verify=False)
    data = json.loads(r.text)
    return data

In [37]:
#気象データ検索関数

def api_search(data,date,key):
    for val in data.values():
        result = str(val[date][key])
    return result

In [ ]:
#CSV出力

with open(str('List') +'.csv', 'w',newline="") as f:
    writer = csv.writer(f)
    no = 0
    writer.writerow(('No','Title','軌道','日付','合計降水量','合計降雪量','日照時間','前日の天気概要（昼）',
                     '前日の天気概況（夜）','当日の天気概況（昼）','当日の天気概況（夜）','WKT'))
    
    for i in range(len(products)):
        title = products_gdf_sorted.iloc[i]["title"]
        orbitdirection = products_gdf_sorted.iloc[i]["orbitdirection"]
        
        if 'GRDH' in title:
            geometry = products_gdf_sorted.iloc[i]["geometry"]
            beginposition = products_gdf_sorted.iloc[i]["beginposition"]
            dt = beginposition.tz_localize('GMT').tz_convert('Asia/Tokyo')
            day = datetime.timedelta(days=1)
            dt_y = dt - day
            
            if no != 0:
                if str(dt.year)+str(dt.month) != str(year) + str(month):
                    data = api_call(area,dt)
                    
                if str(year_y) + str(month_y) != str(dt_y.year)+str(dt_y.month):
                    data_y = api_call(area,dt_y)
                    
            else:
                data = api_call(area,dt)
                data_y = api_call(area,dt_y)
            
            year = dt.year
            month = dt.month
            day = dt.day
            
            year_y = dt_y.year
            month_y = dt_y.month
            day_y = dt_y.day
            
            date = str(year) + '-' + str(month) + '-' + str(day)
            date_y = str(year_y) + '-' + str(month_y) + '-' + str(day_y)
            
            no = no + 1
            
            total_rainfall = api_search(data,date,'合計降水量')
            total_snowfall = api_search(data,date,'合計降雪')
            daylight_hours = api_search(data,date,'日照時間')
            weather_overview_day_y = api_search(data_y,date_y,'天気概況（昼）')
            weather_overview_night_y = api_search(data_y,date_y,'天気概況（夜）')
            weather_overview_day = api_search(data,date,'天気概況（昼）')
            weather_overview_night = api_search(data,date,'天気概況（夜）')            

            print(no, title, orbitdirection, date, total_rainfall, total_snowfall, daylight_hours, weather_overview_day_y,
                  weather_overview_night_y, weather_overview_day, weather_overview_night, geometry)
            
            writer.writerow((no, title, orbitdirection, date, total_rainfall, total_snowfall, daylight_hours, weather_overview_day_y,
                             weather_overview_night_y, weather_overview_day, weather_overview_night, geometry))        
        
f.close()

C:\Users\Shingo Yoshino\anaconda3\lib\site-packages\urllib3\connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.cultivationdata.net'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\Users\Shingo Yoshino\anaconda3\lib\site-packages\urllib3\connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.cultivationdata.net'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


1 S1A_IW_GRDH_1SDV_20170412T205042_20170412T205111_016116_01A9D3_CA76 DESCENDING 2017-4-13 -- -- 10.0 曇一時雨 晴一時曇 晴 快晴 POLYGON ((139.309296 36.658085, 136.518906 37.053249, 136.895737 38.794342, 139.75296 38.400478, 139.309296 36.658085))
2 S1A_IW_GRDH_1SDV_20170412T205111_20170412T205136_016116_01A9D3_AF30 DESCENDING 2017-4-13 -- -- 10.0 曇一時雨 晴一時曇 晴 快晴 POLYGON ((138.955963 35.153004, 136.219696 35.54995, 136.519882 37.053032, 139.309265 36.657993, 138.955963 35.153004))
3 S1A_IW_GRDH_1SDV_20170424T205042_20170424T205111_016291_01AF33_959F DESCENDING 2017-4-25 0.0 -- 9.8 快晴 曇一時晴 薄曇一時晴 曇 POLYGON ((139.309021 36.658142, 136.518631 37.05328, 136.895432 38.794373, 139.75267 38.400536, 139.309021 36.658142))
4 S1A_IW_GRDH_1SDV_20170424T205111_20170424T205136_016291_01AF33_054E DESCENDING 2017-4-25 0.0 -- 9.8 快晴 曇一時晴 薄曇一時晴 曇 POLYGON ((138.95575 35.153049, 136.219589 35.549957, 136.519714 37.053047, 139.309006 36.658051, 138.95575 35.153049))


C:\Users\Shingo Yoshino\anaconda3\lib\site-packages\urllib3\connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.cultivationdata.net'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\Users\Shingo Yoshino\anaconda3\lib\site-packages\urllib3\connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.cultivationdata.net'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


5 S1A_IW_GRDH_1SDV_20170506T205043_20170506T205112_016466_01B479_5B45 DESCENDING 2017-5-7 -- -- 9.4 　雨時々曇 曇一時雨 晴時々曇 快晴 POLYGON ((139.309189 36.658058, 136.518692 37.053257, 136.895584 38.794621, 139.752945 38.400719, 139.309189 36.658058))
6 S1A_IW_GRDH_1SDV_20170506T205112_20170506T205137_016466_01B479_13BD DESCENDING 2017-5-7 -- -- 9.4 　雨時々曇 曇一時雨 晴時々曇 快晴 POLYGON ((138.955856 35.152973, 136.219589 35.549942, 136.519775 37.053024, 139.309158 36.657967, 138.955856 35.152973))
7 S1A_IW_GRDH_1SDV_20170518T205044_20170518T205113_016641_01B9D3_6930 DESCENDING 2017-5-19 -- -- 12.9 曇一時雨後晴 晴 快晴 快晴 POLYGON ((139.309814 36.658089, 136.51944 37.053288, 136.896393 38.794827, 139.753647 38.400925, 139.309814 36.658089))
8 S1A_IW_GRDH_1SDV_20170518T205113_20170518T205138_016641_01B9D3_B4F4 DESCENDING 2017-5-19 -- -- 12.9 曇一時雨後晴 晴 快晴 快晴 POLYGON ((138.956467 35.153004, 136.220215 35.549988, 136.520416 37.05307, 139.309799 36.657997, 138.956467 35.153004))
9 S1A_IW_GRDH_1SDV_20170530T205044_20170530T20

C:\Users\Shingo Yoshino\anaconda3\lib\site-packages\urllib3\connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.cultivationdata.net'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\Users\Shingo Yoshino\anaconda3\lib\site-packages\urllib3\connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.cultivationdata.net'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


...

C:\Users\Shingo Yoshino\anaconda3\lib\site-packages\urllib3\connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.cultivationdata.net'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\Users\Shingo Yoshino\anaconda3\lib\site-packages\urllib3\connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.cultivationdata.net'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


159 S1A_IW_GRDH_1SDV_20191104T205104_20191104T205133_029766_03649A_B176 DESCENDING 2019-11-5 -- -- 9.5 曇一時雨後時々晴 晴 晴 晴 MULTIPOLYGON (((139.31044 36.657818, 139.754135 38.399857, 136.896591 38.793827, 136.519745 37.053085, 139.31044 36.657818)))
160 S1A_IW_GRDH_1SDV_20191104T205133_20191104T205158_029766_03649A_DAC5 DESCENDING 2019-11-5 -- -- 9.5 曇一時雨後時々晴 晴 晴 晴 MULTIPOLYGON (((138.956589 35.152809, 139.310425 36.65773, 136.520721 37.052868, 136.220047 35.549843, 138.956589 35.152809)))
161 S1A_IW_GRDH_1SDV_20191116T205103_20191116T205132_029941_036ABF_03E3 DESCENDING 2019-11-17 0.0 -- 4.2 曇一時晴後時々雨 晴一時曇 曇時々雨一時晴 曇後晴 MULTIPOLYGON (((139.310104 36.657829, 139.75383 38.400047, 136.896286 38.794006, 136.519409 37.053085, 139.310104 36.657829)))
162 S1A_IW_GRDH_1SDV_20191116T205132_20191116T205157_029941_036ABF_79CF DESCENDING 2019-11-17 0.0 -- 4.2 曇一時晴後時々雨 晴一時曇 曇時々雨一時晴 曇後晴 MULTIPOLYGON (((138.956726 35.152744, 139.310089 36.657742, 136.520386 37.052868, 136.220154 35.549782, 138.956726 35.1527

C:\Users\Shingo Yoshino\anaconda3\lib\site-packages\urllib3\connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.cultivationdata.net'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\Users\Shingo Yoshino\anaconda3\lib\site-packages\urllib3\connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.cultivationdata.net'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


165 S1A_IW_GRDH_1SDV_20191210T205103_20191210T205132_030291_0376D6_9D30 DESCENDING 2019-12-11 -- -- 4.8 晴 晴時々曇 曇時々晴、霧を伴う 晴時々曇 MULTIPOLYGON (((139.310608 36.657875, 139.754333 38.400272, 136.896667 38.794205, 136.519775 37.053104, 139.310608 36.657875)))
166 S1A_IW_GRDH_1SDV_20191210T205132_20191210T205157_030291_0376D6_6F08 DESCENDING 2019-12-11 -- -- 4.8 晴 晴時々曇 曇時々晴、霧を伴う 晴時々曇 MULTIPOLYGON (((138.956787 35.152859, 139.310577 36.657784, 136.520752 37.052883, 136.220123 35.549854, 138.956787 35.152859)))
167 S1A_IW_GRDH_1SDV_20191222T205102_20191222T205131_030466_037CE1_CF28 DESCENDING 2019-12-23 1.5 -- 6.2 曇一時晴後一時雨 みぞれ時々雨一時曇 晴時々曇一時みぞれ 雨時々曇一時晴、みぞれを伴う MULTIPOLYGON (((139.310837 36.657864, 139.75473 38.400799, 136.897034 38.79475, 136.52002 37.053112, 139.310837 36.657864)))
168 S1A_IW_GRDH_1SDV_20191222T205131_20191222T205156_030466_037CE1_F904 DESCENDING 2019-12-23 1.5 -- 6.2 曇一時晴後一時雨 みぞれ時々雨一時曇 晴時々曇一時みぞれ 雨時々曇一時晴、みぞれを伴う MULTIPOLYGON (((138.956955 35.152859, 139.310806 36.657772, 136.52099

C:\Users\Shingo Yoshino\anaconda3\lib\site-packages\urllib3\connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.cultivationdata.net'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [17]:
pprint.pprint(data_y)

{'長野': {'2018-6-1': {'合計降水量': '0.0',
                     '合計降雪': '--',
                     '天気概況（夜）': '快晴',
                     '天気概況（昼）': '晴一時曇',
                     '平均気温': '15.7',
                     '平均湿度': '76',
                     '平均風速': '2.3',
                     '日照時間': '5.3',
                     '最低気温': '12.2',
                     '最大10分間降水量': '0.0',
                     '最大1時間降水量': '0.0',
                     '最大瞬間風速': '9.2',
                     '最大瞬間風速（風向）': '東北東',
                     '最大風速': '5.8',
                     '最大風速（風向）': '東',
                     '最小湿度': '51',
                     '最深積雪': '--',
                     '最高気温': '21.8',
                     '海面気圧（平均）': '1012.6',
                     '現地気圧（平均）': '964.0'},
        '2018-6-10': {'合計降水量': '0.0',
                      '合計降雪': '--',
                      '天気概況（夜）': '曇時々雨',
                      '天気概況（昼）': '曇',
                      '平均気温': '20.0',
                      '平均湿度': '71',
              